In [1]:
# notebooks/02_Quantitative_Analysis.ipynb

import os
import sys

# --- Corrected way to find project root in a Jupyter Notebook ---
# This function tries to find the project root by looking for
# specific known directories like 'src', 'data', 'notebooks'.
def find_project_root(current_path):
    path = current_path
    while path != os.path.dirname(path): # Traverse up until the filesystem root
        # Check if this path contains the expected project structure
        if (os.path.isdir(os.path.join(path, 'src')) and
            os.path.isdir(os.path.join(path, 'data')) and
            os.path.isdir(os.path.join(path, 'notebooks'))):
            return path # Found the project root
        path = os.path.dirname(path) # Move up to the parent directory
    return current_path # Fallback: if structure not found, return original path

# Get the current working directory (where the notebook was launched from)
current_working_dir = os.getcwd()

# Find the project root using the helper function
project_root = find_project_root(current_working_dir)

# Add the project root to sys.path
# This allows Python to find 'src' as a top-level module
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added '{project_root}' to sys.path for module imports.")
else:
    print(f"'{project_root}' already in sys.path.")

# --- Now you can safely import from src.config ---
from src.config import STOCK_DATA_DIR, STOCK_TICKERS

Added 'c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-1\NewsSentiment-StockPrice-Prediction' to sys.path for module imports.
Project structure setup complete and config.py created/updated.
Base Directory: c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-1\NewsSentiment-StockPrice-Prediction
News Raw Path: c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-1\NewsSentiment-StockPrice-Prediction\data\raw_analyst_ratings.csv
Stock Data Directory: c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-1\NewsSentiment-StockPrice-Prediction\data\yfinance_data
Tickers to process: ['AAPL', 'AMZN', 'GOOG', 'META', 'NVDA', 'TSLA']


# Import Necessary Libraries

In [2]:
import pandas as pd
import yfinance as yf # For fetching financial data
import talib as ta    # For technical analysis indicators
import numpy as np    # For numerical operations
import matplotlib.pyplot as plt # For plotting
import mplfinance as mpf # For advanced financial charts
import pynance as pn # For financial metrics (as per task requirement)

# Load  and prepare the data 

In [3]:

# --- 2. Load and Prepare ALL Stock Data ---
print("\n--- Loading and Preparing Data for all Tickers ---")

all_stocks_df = {} # This dictionary will store DataFrames for each ticker

for ticker in STOCK_TICKERS:
    # Construct the full file path using os.path.join with the correct filename suffix
    file_name = f"{ticker}_historical_data.csv" # Corrected filename
    file_path = os.path.join(STOCK_DATA_DIR, file_name)

    print(f"Attempting to load data for {ticker} from: {file_path}")
    try:
        df_ticker = pd.read_csv(file_path, parse_dates=True, index_col='Date')
        
        # Standardize column names: replace spaces and ensure 'Close' column
        df_ticker.columns = [col.replace(' ', '_') for col in df_ticker.columns]
        if 'Adj_Close' in df_ticker.columns and 'Close' not in df_ticker.columns:
            df_ticker['Close'] = df_ticker['Adj_Close']
        
        # Ensure all required OHLCV columns are present
        required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        if not all(col in df_ticker.columns for col in required_cols):
            print(f"\nWARNING: Missing one or more required OHLCV columns for {ticker}.")
            print(f"Found: {df_ticker.columns.tolist()}. Required: {required_cols}")
            # Skip this ticker if critical columns are missing
            continue 

        # Drop rows with any NaN values
        df_ticker.dropna(inplace=True) 
        
        if not df_ticker.empty:
            all_stocks_df[ticker] = df_ticker
            print(f"Data loaded successfully for {ticker}. Shape: {df_ticker.shape}")
        else:
            print(f"WARNING: DataFrame for {ticker} is empty after cleaning. Skipping.")

    except FileNotFoundError:
        print(f"ERROR: Local data file not found for {ticker} at {file_path}. Skipping this ticker.")
    except Exception as e:
        print(f"ERROR: An unexpected error occurred loading {ticker} data: {e}. Skipping this ticker.")

if not all_stocks_df:
    sys.exit("Exiting: No stock data loaded successfully for any ticker.")

print(f"\nSuccessfully loaded data for {len(all_stocks_df)} out of {len(STOCK_TICKERS)} tickers.")
print("Loaded tickers:", list(all_stocks_df.keys()))


# --- Iterate through each stock for analysis and visualization ---
print("\n--- Performing Analysis and Visualization for Each Ticker ---")

for ticker, df in all_stocks_df.items():
    print(f"\n--- Analyzing {ticker} ---")

    # Ensure 'Close' column is numeric for TA-Lib operations
    if not pd.api.types.is_numeric_dtype(df['Close']):
        df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
        df.dropna(subset=['Close'], inplace=True)
        if df.empty:
            print(f"Skipping {ticker}: 'Close' column became empty after numeric conversion.")
            continue


--- Loading and Preparing Data for all Tickers ---
Attempting to load data for AAPL from: c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-1\NewsSentiment-StockPrice-Prediction\data\yfinance_data\AAPL_historical_data.csv
Data loaded successfully for AAPL. Shape: (10998, 8)
Attempting to load data for AMZN from: c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-1\NewsSentiment-StockPrice-Prediction\data\yfinance_data\AMZN_historical_data.csv
Data loaded successfully for AMZN. Shape: (6846, 8)
Attempting to load data for GOOG from: c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-1\NewsSentiment-StockPrice-Prediction\data\yfinance_data\GOOG_historical_data.csv
Data loaded successfully for GOOG. Shape: (5020, 8)
Attempting to load data for META from: c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-1\NewsSentiment-StockPrice-Prediction\data\yfinance_data\META_historical_data.csv
Data loaded successfully for META. Shape: (2926, 8)
Attempting to load data for NVDA from: c:\Users\hp\OneDrive\Desktop\kaim-ai\KAIM-1\News

In [ ]:

    # --- 3. Apply Analysis Indicators with TA-Lib ---
    # Simple Moving Averages (SMA)
df['SMA_10'] = ta.SMA(df['Close'], timeperiod=10)
df['SMA_50'] = ta.SMA(df['Close'], timeperiod=50)
df['SMA_200'] = ta.SMA(df['Close'], timeperiod=200)

    # Relative Strength Index (RSI)
df['RSI'] = ta.RSI(df['Close'], timeperiod=14)

    # Moving Average Convergence Divergence (MACD)
macd, macdsignal, macdhist = ta.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
df['MACD'] = macd
df['MACD_Signal'] = macdsignal
df['MACD_Hist'] = macdhist

    # Bollinger Bands (BBANDS)
upperband, middleband, lowerband = ta.BBANDS(df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
df['BB_Upper'] = upperband
df['BB_Middle'] = middleband
df['BB_Lower'] = lowerband

    # Stochastic Oscillator (STOCH) - Requires High, Low, Close
if all(col in df.columns for col in ['High', 'Low']):
        slowk, slowd = ta.STOCH(df['High'], df['Low'], df['Close'],
                                fastk_period=14, slowk_period=3, slowk_matype=0,
                                slowd_period=3, slowd_matype=0)
        df['Stoch_K'] = slowk
        df['Stoch_D'] = slowd
else:
        print(f"WARNING ({ticker}): High/Low columns not found. Skipping Stochastic Oscillator calculation.")

    # Average Directional Movement Index (ADX) - Requires High, Low, Close
if all(col in df.columns for col in ['High', 'Low']):
        df['ADX'] = ta.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)
else:
        print(f"WARNING ({ticker}): High/Low columns not found. Skipping ADX calculation.")

    # Drop rows with NaN values that result from indicator calculations (these are typically at the beginning of the series)
df.dropna(inplace=True)
if df.empty:
    print(f"Skipping {ticker}: DataFrame became empty after TA-Lib indicator calculation and NaNs dropped.")
    continue
print(f"DataFrame shape for {ticker} after TA-Lib indicator calculation and NaNs dropped: {df.shape}")
    # print(df.tail()) # Uncomment to see tail for each ticker


--- Applying Technical Analysis Indicators with TA-Lib ---
DataFrame shape after TA-Lib indicator calculation and NaNs dropped: (6647, 21)

DataFrame with Technical Indicators (last 5 rows):
                  Open        High         Low       Close   Adj_Close  \
Date                                                                     
2024-07-24  183.199997  185.449997  180.410004  180.830002  180.830002   
2024-07-25  182.910004  183.899994  176.800003  179.850006  179.850006   
2024-07-26  180.389999  183.190002  180.240005  182.500000  182.500000   
2024-07-29  183.839996  184.750000  182.380005  183.199997  183.199997   
2024-07-30  184.720001  185.860001  179.380005  181.710007  181.710007   

              Volume  Dividends  Stock_Splits      SMA_10      SMA_50  ...  \
Date                                                                   ...   
2024-07-24  41532400        0.0           0.0  187.988002  187.364001  ...   
2024-07-25  44464200        0.0           0.0  186.4680